In [271]:
import random
import time #not used
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split 
from functools import reduce #not used
from sklearn.metrics import classification_report 
from sklearn import preprocessing #not used
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler #not used
import keras
from keras.models import Sequential #not used
from keras.layers import Dense #not used
from keras.layers import LeakyReLU #not used
from keras.layers import Dropout #not used
import tensorflow as tf #not used
from tensorflow.keras import layers

In [272]:
# global variables
safe_action = []
safe_result = []
safe_save_money = []
safe_earning = []
safe_player_initial_sum = []
safe_player_final_sum = []
safe_dealer_first_card = []
safe_has_ace = []
safe_correct_action = []


random_action = []
random_result = []
random_save_money = []
random_earning = []
random_player_initial_sum = []
random_player_final_sum = []
random_dealer_first_card = []
random_has_ace = []
random_correct_action = []

AI_possible_action = []
AI_result = []
AI_save_money = []
AI_earning = []
AI_initial_sum = []
AI_dealer_first_card = []
AI_has_ace = []
AI_actual_action = []


safe_win_loss_record = []
random_win_loss_record = []
AI_win_loss_record = []

playing = True
deck_cards = []

In [273]:
# Basic definition of the cards. Simple as it is
class deck_of_cards:
  def __init__(self, card_suits, card_characters, card_point_value):
    self.card_suits = card_suits
    self.card_characters = card_characters
    self.card_point_value = card_point_value

In [274]:
# Add a hidden function for the dealer second card in here some how
# Maybe use a boolean for the hidden card
def display_cards(cards):
    display_top_wall = ""
    for card in cards:
        display_top_wall = display_top_wall + "  ______________  "
    print(display_top_wall)
 

    display_vertical_wall = ""
    for card in cards:
        display_vertical_wall = display_vertical_wall + " |              | "
    print(display_vertical_wall)
 
    display_card_character = ""
    for card in cards:
        # if it is a 10, remove one of the spacing
        if card.card_characters == '10':
            display_card_character = display_card_character + " |  {}          | ".format(card.card_characters)
        # if not a 10, keep the normal spacing
        else:
            display_card_character = display_card_character + " |  {}           | ".format(card.card_characters)  
    print(display_card_character)

    for i in range (1,4):
        display_vertical_wall = ""
        for card in cards:
            display_vertical_wall = display_vertical_wall + " |              | "
        print(display_vertical_wall)
 
    display_card_suit = ""
    for card in cards:
        display_card_suit = display_card_suit + " |      {}      | ".format(card.card_suits)
    print(display_card_suit)    

    for i in range (1,4):
        display_vertical_wall = ""
        for card in cards:
            display_vertical_wall = display_vertical_wall + " |              | "
        print(display_vertical_wall)

    display_card_character = ""
    for card in cards:
        # if it is a 10, remove one of the spacing
        if card.card_characters == '10':
            display_card_character = display_card_character + " |          {}  | ".format(card.card_characters)
        # if not a 10, keep the normal spacing
        else:
            display_card_character = display_card_character + " |           {}  | ".format(card.card_characters)  
    print(display_card_character)  
         
    display_bottom_wall = ""
    for card in cards:
        display_bottom_wall = display_bottom_wall + " |______________| "
    print(display_bottom_wall)       
  
  

In [275]:
def blackjack_rules():
  print("Rules of blackjack for the players are as follows: ")
  print("Players must have a higher score than the dealer, but cannot exceed 21")
  print("Players can either choose to 'hit' to get another card or 'stand' to keep their score")
  print("If the player gets 21, the player automatically wins.")
  print("If the player exceeds 21, the player bust and automatically loses.")
  print("If the player has the same score as the dealer, the game is a tie.")
  print("Players must enter a bet before the blackjack game can start.")
  
  print("\nRules of blackjack for the dealers are as follows:")
  print("The dealer must hit until their score exceeds 16. If dealer score is over 16, the dealer must stand.")
  print("If the dealer gets 21, the dealer automatically wins.")
  print("If the dealer exceeds 21, the player bust and automatically loses.")
  print("If the dealer has the same score as the dealer, the game is a tie.")

In [276]:
def shuffle():
  card_suits = ["Hearts", "Diamonds", "Clubs", "Spades"] # There is only 4 suits in a deck of cards
  suits_graphics = {"Hearts":"\u2661", "Diamonds": "\u2662", "Clubs": "\u2667", "Spades":"\u2664"} # Graphics of the suits looks better
  card_characters = ["A", "2", "3", "4", "5", "6", "7", "8", "9", "10", "J", "Q", "K"] # All possible cards characters in a deck

  # Assigns the point value to the characters of the cards 
  card_point_value = {"A": 1, "2": 2, "3": 3, "4": 4, "5": 5, "6": 6, "7": 7, "8": 8, "9": 9, "10": 10, "J": 10, "Q": 10, "K": 10}
  deck_cards = [] # Only using 4 decks of cards

  # Sets up our 2 deck of cards
  for i in range(1,5):
    for suit in card_suits:
      for card in card_characters:
        deck_cards.append(deck_of_cards(suits_graphics[suit], card, card_point_value[card])) # 4 deck = 208 cards
  #print("\nReshuffling 4 decks of cards Please wait!\n")
  return deck_cards
#print(len(deck_cards)) # 1 deck = 52 cards, 2 deck = 104 cards, etc... 
#blackjack_rules()

In [277]:
def total_check(hand):
  #total of cards in hand
  sum = 0
  #keeps track on the number of ace cards counting as 11
  ace_count = 0
  for card in hand:
    sum += card.card_point_value
    #changes an ace to count for 11 if doing so would not go over 21 
    if card.card_point_value == 1:
      if sum < 12:
        sum += 10
        ace_count += 1
    # changes an ace back to counting for 1 if the total is over 21 and an ace is being treated as 11
    if sum > 21 and ace_count > 0:
      sum -= 10
      ace_count -= 1
  return sum

In [278]:
def blackjack_check(hand, hand_sum):
  if len(hand) == 2 and hand_sum == 21:
    return True
  else:
    return False

In [279]:
def safe_move(deck, player_cards, bet):
  playing = True
  split = False #not used
  sum = total_check(player_cards)
  while playing == True:
    if sum <= 11 :
      #print("HIT")
      new_card = random.choice(deck)
      player_cards.append(new_card)
      deck.remove(new_card)
      sum = total_check(player_cards)
      #print("\nPlayer Cards:")
      #display_cards(player_cards)
      player_total = total_check(player_cards) #not used
      #print("Player's Total:", player_total,)
      safe_action.append(1)
      if sum > 21:
        #print("Bust.")
        playing = False
        break
      if sum == 21:
        playing = False
        break
      break
    elif sum > 11:
      safe_action.append(0)
      #print("STAY")
      playing = False
      break
      """
    elif sum == 11:
      #print("Double Down")
      bet = bet * 2
      new_card = random.choice(deck)
      player_cards.append(new_card)
      deck.remove(new_card)
      sum = total_check(player_cards)
      #print("\nPlayer Cards:")
      #display_cards(player_cards)
      player_total = total_check(player_cards)
      #print("Player's Total:", player_total)
      safe_action.append(3)
      if sum > 21:
        #print("Bust.")
        playing = False
      playing = False
      break"""
  return deck, player_cards, bet, sum

In [280]:
def random_move(deck, player_cards, bet):
  playing = True
  split = False #not used
  sum = total_check(player_cards)
  import random
  get_random_action = random.uniform(0,1)
  #print(get_random_action)
  while playing == True:
    # If action is 0.65 or greater, hit
    if get_random_action >= 0.55 :
      #print("HIT")
      new_card = random.choice(deck)
      player_cards.append(new_card)
      deck.remove(new_card)
      sum = total_check(player_cards)
      #print("\nPlayer Cards:")
      #display_cards(player_cards)
      player_total = total_check(player_cards) #not used
      #print("Player's Total:", player_total,)
      random_action.append(1)
      if sum > 21:
        #print("Bust.")
        playing = False
        break
      if sum == 21:
        playing = False
        break
      break
    # if action is below 0.55 the action will stay
    elif get_random_action < 0.55:
      random_action.append(0)
      #print("STAY")
      playing = False
      break
    # if action is below 0.29, the action will double down
    """
    elif get_random_action < 0.30:
      #print("Double Down")
      bet = bet * 2
      new_card = random.choice(deck)
      player_cards.append(new_card)
      deck.remove(new_card)
      sum = total_check(player_cards)
      #print("\nPlayer Cards:")
      #display_cards(player_cards)
      player_total = total_check(player_cards)
      #print("Player's Total:", player_total)
      random_action.append(3)
      if sum > 21:
        #print("Bust.")
        playing = False
      playing = False
      break"""
  return deck, player_cards, bet, sum

In [281]:
def AI_game_move(deck, player_cards, bet, AI_action):
  playing = True
  split = False #not used
  sum = total_check(player_cards)
  while playing == True:
    choice = AI_action
    if choice == 1:
      new_card = random.choice(deck)
      player_cards.append(new_card)
      deck.remove(new_card)
      sum = total_check(player_cards)
      #print("\nPlayer Cards:")
      #display_cards(player_cards)
      player_total = total_check(player_cards) #not used
      #print("Player's Total:", player_total,"\n")
      if sum > 21:
        #print("Bust.")
        playing = False
        break
      if sum == 21:
        playing = False
        break
    elif choice == 0:
      playing = False
      break
  return deck, player_cards, bet, sum

In [282]:
def dealer_move(deck, dealer_cards,computer_play):
  total = 0
  dealer_total = total_check(dealer_cards)
  while dealer_total < 17:
    new_card = random.choice(deck)
    dealer_cards.append(new_card)
    deck.remove(new_card)
    if(computer_play == False):
      #print("Dealer Cards:")
      display_cards(dealer_cards)
    ace_count = 0
    for card in dealer_cards:
      total += card.card_point_value
      #changes an ace to count for 11 if doing so would not go over 21 
      if card.card_point_value == 1:
        if total < 12:
          total += 10
          ace_count += 1
      # changes an ace back to counting for 1 if the total is over 21 and an ace is being treated as 11
      if total > 21 and ace_count > 0:
        total -= 10
        ace_count -= 1
    #After the cards in hand have been counted and the total is 17 and an ace counts as 11, make the ace count as 1 to keep adding cards
    if total == 17 and ace_count > 0:
      total -= 10
      ace_count -= 1
    sum = total_check(dealer_cards) #not used
    dealer_total = total_check(dealer_cards)
    #print("\nDealer's Total:", dealer_total,"\n")
  return deck, dealer_cards, dealer_total



In [283]:
def safe_game(player_money, deck_cards, computer_play):
  bet = 5
  # Deals the player a random card.
  player_cards = []
  dealer_cards = []
  #player_card = random.choice(deck_cards)
  #player_cards.append(player_card)
  #deck.remove(player_card)

  safe_play_win = [] #not used

  player_bust = False
  dealer_bust = False
  count = 0
  # Need to have this display both cards one at a time. Having issues with it atm
  while len(player_cards) < 2:
    # Deals the player a random card.
    deal_player_card = random.choice(deck_cards)
    player_cards.append(deal_player_card)
    deck_cards.remove(deal_player_card)
    # Deal the dealer a random card.
    deal_dealer_card = random.choice(deck_cards)
    dealer_cards.append(deal_dealer_card)
    deck_cards.remove(deal_dealer_card)
    # Gets the first card of the dealer.
    if count == 0:
      dealer_card_1 = total_check(dealer_cards)
      safe_dealer_first_card.append(dealer_card_1)
      count += 1

  # If player has an ace, place a 1 in the dataframe.
  ggggg = 0
  for card in player_cards:
    ggggg += 1
    new_card = card.card_point_value
    if(new_card == 1):
      safe_has_ace.append(1)
      break
    if(ggggg == 2 and new_card != 1):
      safe_has_ace.append(0)

  if(computer_play == False):
    print("Dealer Cards:")
    display_cards(dealer_cards)

  dealer_total = total_check(dealer_cards)
  #print("\nDealer's Total:", dealer_total,"\n")

  if(computer_play == False):
    print("Player Cards:")
    display_cards(player_cards)

  player_total = total_check(player_cards)
  #print("Player's Total:", player_total)
  safe_player_initial_sum.append(player_total)

  player_blackjack = blackjack_check(player_cards, player_total)
  dealer_blackjack = blackjack_check(dealer_cards, dealer_total)

  if dealer_blackjack == True and player_blackjack == False:
    #print("Blackjack! Dealer wins!")
    player_money -= bet
    safe_action.append(0)
    safe_result.append(0)

  elif dealer_blackjack == False and player_blackjack == True:
    #print("Blackjack! Player wins!")
    player_money += bet * (3/2)
    safe_action.append(0)
    safe_result.append(1)

  elif dealer_blackjack == True and player_blackjack == True:
    #print("Two blackjacks! Push!")
    safe_action.append(0)
    safe_result.append(2)

  else:
    deck_cards, player_cards, bet,  player_total = safe_move(deck_cards, player_cards, bet)
    deck_cards, dealer_cards, dealer_total = dealer_move(deck_cards, dealer_cards, computer_play)


    if (player_total > 21):
      #print("Player bust! Dealer wins! Player total:", player_total)
      player_bust = True
      player_money = player_money - bet
      safe_result.append(0)
      

    elif (dealer_total > 21):
      #print("Dealer bust! Player wins! Dealer total:", dealer_total)
      dealer_bust = True
      player_money = player_money + bet
      safe_result.append(1)
      

    if (player_bust == False and dealer_bust == False and player_total > dealer_total): 
      #print("Player wins!")
      player_money += bet
      safe_result.append(1)

    elif (player_bust == False and dealer_bust == False and player_total < dealer_total):
      #print("Dealer wins!")
      player_money -= bet
      safe_result.append(0)

    elif (player_bust == False and dealer_bust == False and player_total == dealer_total):
      #print("It's a push!")
      safe_result.append(2)

  #print("THIS IS PLAYER TOTAL:", player_total)
  #print("THIS IS DEALER TOTAL:", dealer_total)
  safe_player_final_sum.append(player_total)

  card_number_check = len(deck_cards)
  #print("This is amount of cards left:", card_number_check)

  if card_number_check < 20:
    deck_cards = shuffle()
  return player_money, deck_cards

In [284]:
def random_game(player_money, deck_cards, computer_play):
  bet = 5
  # Deals the player a random card.
  player_cards = []
  dealer_cards = []
  #player_card = random.choice(deck_cards)
  #player_cards.append(player_card)
  #deck.remove(player_card)

  random_play_win = [] #not used

  player_bust = False
  dealer_bust = False
  count = 0
  # Need to have this display both cards one at a time. Having issues with it atm
  while len(player_cards) < 2:
    # Deals the player a random card.
    deal_player_card = random.choice(deck_cards)
    player_cards.append(deal_player_card)
    deck_cards.remove(deal_player_card)
    # Deal the dealer a random card.
    deal_dealer_card = random.choice(deck_cards)
    dealer_cards.append(deal_dealer_card)
    deck_cards.remove(deal_dealer_card)
    # Gets the first card of the dealer.
    if count == 0:
      dealer_card_1 = total_check(dealer_cards)
      random_dealer_first_card.append(dealer_card_1)
      count += 1

  # If player has an ace, place a 1 in the dataframe.
  ggggg = 0
  for card in player_cards:
    ggggg += 1
    new_card = card.card_point_value
    if(new_card == 1):
      random_has_ace.append(1)
      break
    if(ggggg == 2 and new_card != 1):
      random_has_ace.append(0)

  if(computer_play == False):
    print("Dealer Cards:")
    display_cards(dealer_cards)

  dealer_total = total_check(dealer_cards)
  #print("\nDealer's Total:", dealer_total,"\n")

  if(computer_play == False):
    print("Player Cards:")
    display_cards(player_cards)

  player_total = total_check(player_cards)
  #print("Player's Total:", player_total)
  random_player_initial_sum.append(player_total)

  player_blackjack = blackjack_check(player_cards, player_total)
  dealer_blackjack = blackjack_check(dealer_cards, dealer_total)

  if dealer_blackjack == True and player_blackjack == False:
    #print("Blackjack! Dealer wins!")
    player_money -= bet
    random_action.append(0)
    random_result.append(0)

  elif dealer_blackjack == False and player_blackjack == True:
    #print("Blackjack! Player wins!")
    player_money += bet * (3/2)
    random_action.append(0)
    random_result.append(1)

  elif dealer_blackjack == True and player_blackjack == True:
    #print("Two blackjacks! Push!")
    random_action.append(0)
    random_result.append(2)

  else:
    # Insert a way to have the model predict the action to choose from depending on player card, dealer card, action, and earning.
    deck_cards, player_cards, bet,  player_total = random_move(deck_cards, player_cards, bet)
    deck_cards, dealer_cards, dealer_total = dealer_move(deck_cards, dealer_cards, computer_play)


    if (player_total > 21):
      #print("Player bust! Dealer wins! Player total:", player_total)
      player_bust = True
      player_money = player_money - bet
      random_result.append(0)
      

    elif (dealer_total > 21):
      #print("Dealer bust! Player wins! Dealer total:", dealer_total)
      dealer_bust = True
      player_money = player_money + bet
      random_result.append(1)
      

    if (player_bust == False and dealer_bust == False and player_total > dealer_total): 
      #print("Player wins!")
      player_money += bet
      random_result.append(1)

    elif (player_bust == False and dealer_bust == False and player_total < dealer_total):
      #print("Dealer wins!")
      player_money -= bet
      random_result.append(0)

    elif (player_bust == False and dealer_bust == False and player_total == dealer_total):
      #print("It's a push!")
      random_result.append(2)

  #print("THIS IS PLAYER TOTAL:", player_total)
  #print("THIS IS DEALER TOTAL:", dealer_total)
  random_player_final_sum.append(player_total)

  card_number_check = len(deck_cards)
  #print("This is amount of cards left:", card_number_check)

  if card_number_check < 20:
    deck_cards = shuffle()
  return player_money, deck_cards

In [285]:
def get_model_prediction(player_hand, dealer_card, has_ace, action):
  AI_df = pd.DataFrame()
  AI_df['Player Initial'] = player_hand
  AI_df['Dealer Hand'] = dealer_card
  AI_df['Has Ace'] = has_ace
  AI_df['Action'] = action
  #print(AI_df)
  scaler = MinMaxScaler()
  # Transforming the data for AI_df using MinMaxScaler
  AI_df_scaler = scaler.fit_transform(AI_df) #not used
  AI_action = blackjack_AI.predict(AI_df)
  AI_action = AI_action.round()
  
  #print("This is AI_Action", AI_action)

  if(AI_action == 1):
    AI_action = 1
  elif(AI_action == 0):
    AI_action = 0
  AI_actual_action.append(AI_action)
  #print("Updated AI Action", AI_action)

  return AI_action

In [286]:
def model_AI_game(player_money, deck_cards, computer_play):
  bet = 5
  # Deals the player a random card.
  player_cards = []
  dealer_cards = []
  #player_card = random.choice(deck_cards)
  #player_cards.append(player_card)
  #deck.remove(player_card)

  AI_action = []
  import random
  get_possible_action = random.uniform(0,1)
  if(get_possible_action >= 0.50):
    AI_action.append(1)
    AI_possible_action.append(1)
  else:
    AI_action.append(0)
    AI_possible_action.append(0)


  player_bust = False
  dealer_bust = False
  count = 0
  # Need to have this display both cards one at a time. Having issues with it atm
  while len(player_cards) < 2:
    # Deals the player a random card.
    deal_player_card = random.choice(deck_cards)
    player_cards.append(deal_player_card)
    deck_cards.remove(deal_player_card)
    # Deal the dealer a random card.
    deal_dealer_card = random.choice(deck_cards)
    dealer_cards.append(deal_dealer_card)
    deck_cards.remove(deal_dealer_card)
    # Gets the first card of the dealer.
    if count == 0:
      dealer_card_1 = total_check(dealer_cards)
      #safe_dealer_first_card.append(dealer_card_1)
      count += 1
      #if(dealer_card_1 == 11):########################################################################################################

      # If player has an ace, place a 1 in the dataframe.
  ggggg = 0
  #AI_has_ace = []
  AI_player_hand = []
  AI_dealer_card = []
  AI_aces = []
  
  for card in player_cards:
    ggggg += 1
    new_card = card.card_point_value
    if(new_card == 1):
      AI_aces.append(1)
      AI_has_ace.append(1)
      break
    if(ggggg == 2 and new_card != 1):
      AI_aces.append(0)
      AI_has_ace.append(0)

  if(computer_play == False):
    print("Dealer Cards:")
    display_cards(dealer_cards)

  dealer_total = total_check(dealer_cards)
  #print("\nDealer's Total:", dealer_total,"\n")

  if(computer_play == False):
    print("Player Cards:")
    display_cards(player_cards)

  player_total = total_check(player_cards)


  AI_dealer_card.append(dealer_card_1)
  AI_player_hand.append(player_total)
  AI_initial_sum.append(player_total)
  AI_dealer_first_card.append(dealer_card_1)

  player_blackjack = blackjack_check(player_cards, player_total)
  dealer_blackjack = blackjack_check(dealer_cards, dealer_total)

  if dealer_blackjack == True and player_blackjack == False:
    #print("Blackjack! Dealer wins!")
    player_money -= bet
    #safe_action.append(0)
    AI_result.append(0)
    AI_actual_action.append(0)

  elif dealer_blackjack == False and player_blackjack == True:
    #print("Blackjack! Player wins!")
    player_money += bet * (3/2)
    #safe_action.append(0)
    AI_result.append(1)
    AI_actual_action.append(0)

  elif dealer_blackjack == True and player_blackjack == True:
    #print("Two blackjacks! Push!")
    #safe_action.append(0)
    AI_result.append(2)
    AI_actual_action.append(0)

  else:
    
    super_count = 0
    AI_choice = 1
    while(player_total < 21) and (AI_choice == 1):
      AI_choice = get_model_prediction(AI_player_hand, AI_dealer_card, AI_aces, AI_action)
      deck_cards, player_cards, bet,  player_total = AI_game_move(deck_cards, player_cards, bet, AI_choice)
      #super_count += 1
      #print("This is count for rounds",super_count)
      #print("AI_choice:", AI_choice)
      #print("Player total:", player_total)


    deck_cards, dealer_cards, dealer_total = dealer_move(deck_cards, dealer_cards, computer_play)
    #print("Dealer total:", dealer_total)

    if (player_total > 21):
      #print("Player bust! Dealer wins! Player total:", player_total)
      player_bust = True
      player_money = player_money - bet
      AI_result.append(0)
      

    elif (dealer_total > 21):
      #print("Dealer bust! Player wins! Dealer total:", dealer_total)
      dealer_bust = True
      player_money = player_money + bet
      AI_result.append(1)
      

    if (player_bust == False and dealer_bust == False and player_total > dealer_total): 
      #print("Player wins!")
      player_money += bet
      AI_result.append(1)

    elif (player_bust == False and dealer_bust == False and player_total < dealer_total):
      #print("Dealer wins!")
      player_money -= bet
      AI_result.append(0)

    elif (player_bust == False and dealer_bust == False and player_total == dealer_total):
      #print("It's a push!")
      AI_result.append(2)

  #print("THIS IS PLAYER TOTAL:", player_total)
  #print("THIS IS DEALER TOTAL:", dealer_total)
  #safe_player_final_sum.append(player_total)

  card_number_check = len(deck_cards)
  #print("This is amount of cards left:", card_number_check)

  if card_number_check < 20:
    deck_cards = shuffle()
  return player_money, deck_cards

In [287]:
blackjack_rules()

Rules of blackjack for the players are as follows: 
Players must have a higher score than the dealer, but cannot exceed 21
Players can either choose to 'hit' to get another card or 'stand' to keep their score
If the player gets 21, the player automatically wins.
If the player exceeds 21, the player bust and automatically loses.
If the player has the same score as the dealer, the game is a tie.
Players must enter a bet before the blackjack game can start.

Rules of blackjack for the dealers are as follows:
The dealer must hit until their score exceeds 16. If dealer score is over 16, the dealer must stand.
If the dealer gets 21, the dealer automatically wins.
If the dealer exceeds 21, the player bust and automatically loses.
If the dealer has the same score as the dealer, the game is a tie.


In [288]:
computer_play = True
safe_play_money = 75000000
update_money = 0
count = 1
safe_profit_or_loss = 0
int(safe_play_money)
deck_cards = shuffle()
for i in range(0,170000,1):
  if(safe_play_money < 5):
    break
  if(safe_play_money > safe_play_money*2):
    break
  if(i+1 == (count * 10000)):
    print("Iterated over",i+1,"rounds")
    count += 1
  update_money, deck_cards = safe_game(safe_play_money, deck_cards, computer_play)
  #print("\nPrevious $", safe_play_money)
  #print("Player $", update_money)
  safe_profit_or_loss = update_money - safe_play_money
  safe_play_money = update_money
  safe_save_money.append(update_money)
  safe_earning.append(safe_profit_or_loss)
  #print("This is profit or loss:",safe_profit_or_loss)
  

Iterated over 10000 rounds
Iterated over 20000 rounds
Iterated over 30000 rounds
Iterated over 40000 rounds
Iterated over 50000 rounds
Iterated over 60000 rounds
Iterated over 70000 rounds
Iterated over 80000 rounds
Iterated over 90000 rounds
Iterated over 100000 rounds
Iterated over 110000 rounds
Iterated over 120000 rounds
Iterated over 130000 rounds
Iterated over 140000 rounds
Iterated over 150000 rounds
Iterated over 160000 rounds
Iterated over 170000 rounds


In [289]:
computer_play = True
random_play_money = 75000000
random_update_money = 0
random_profit_or_loss = 0
count = 1
int(random_play_money)
deck_cards = shuffle()
for i in range(0,170000,1):
  if(random_play_money < 5):
    break
  if(random_play_money > random_play_money*2):
    break
  if(i+1 == (count * 10000)):
    print("Iterated over",i+1,"rounds")
    count += 1
  random_update_money, deck_cards = random_game(random_play_money, deck_cards, computer_play)
  #print("\nPrevious $", random_play_money)
  #print("Player $", random_update_money)
  random_profit_or_loss = random_update_money - random_play_money
  random_play_money = random_update_money
  random_save_money.append(random_update_money)
  random_earning.append(random_profit_or_loss)
  #print("This is profit or loss:",random_profit_or_loss)

Iterated over 10000 rounds
Iterated over 20000 rounds
Iterated over 30000 rounds
Iterated over 40000 rounds
Iterated over 50000 rounds
Iterated over 60000 rounds
Iterated over 70000 rounds
Iterated over 80000 rounds
Iterated over 90000 rounds
Iterated over 100000 rounds
Iterated over 110000 rounds
Iterated over 120000 rounds
Iterated over 130000 rounds
Iterated over 140000 rounds
Iterated over 150000 rounds
Iterated over 160000 rounds
Iterated over 170000 rounds


In [290]:
for i in range(0,len(safe_action)):
  get_random_correct_action = random.uniform(0,1)
  # Stay
  if(safe_earning[i] >= 0 and safe_action[i] == 0):
    safe_correct_action.append(safe_action[i])
  # Hit
  elif(safe_earning[i] >= 0 and safe_action[i] == 1):
    safe_correct_action.append(safe_action[i])
  # Stay bad
  elif(safe_earning[i] < 0 and safe_action[i] == 0):
    safe_correct_action.append(1)
  # Hit bad
  elif(safe_earning[i] < 0 and safe_action[i] == 1):
    safe_correct_action.append(0)

  
for i in range(0,len(random_action)):
  # Stay
  if(random_earning[i] >= 0 and random_action[i] == 0):
    random_correct_action.append(random_action[i])
  # Hit
  elif(random_earning[i] >= 0 and random_action[i] == 1):
    random_correct_action.append(random_action[i])
  # Stay bad
  elif(random_earning[i] < 0 and random_action[i] == 0):
    random_correct_action.append(1)
  # Hit bad
  elif(random_earning[i] < 0 and random_action[i] == 1):
    random_correct_action.append(0)

In [291]:
safe_dataframe = pd.DataFrame()
safe_dataframe['Player Initial'] = safe_player_initial_sum
safe_dataframe['Dealer Hand'] = safe_dealer_first_card
safe_dataframe['Has Ace'] = safe_has_ace
safe_dataframe['Action'] = safe_action
safe_dataframe['Earning'] = safe_earning
safe_dataframe['Money'] = safe_save_money
safe_dataframe['Result'] = safe_result
safe_dataframe['Correct Action'] = safe_correct_action

In [292]:
# Player Initial is player starting sum 
# Dealer hand is the first card they get. 
# Action varies. (0: no action, 1: hit, 2: stay, 3: double down)
# Earning varies. Anything negative is a loss, 0's is a push, and anything + is a win
# Money Keeps count of the money. They start with $1000 
# Result varies. (0: lose, 1: win, 2: tie)
safe_dataframe

,Player Initial,Dealer Hand,Has Ace,Action,Earning,Money,Result,Correct Action
0,8,9,0,1,-5.0,74999995.0,0,0
1,13,9,1,0,-5.0,74999990.0,0,1
2,14,6,0,0,-5.0,74999985.0,0,1
3,20,9,0,0,5.0,74999990.0,1,0
4,11,9,0,1,5.0,74999995.0,1,1
...,...,...,...,...,...,...,...,...
169995,11,2,0,1,5.0,74923535.0,1,1
169996,18,10,0,0,-5.0,74923530.0,0,1
169997,13,6,0,0,-5.0,74923525.0,0,1
169998,16,4,0,0,-5.0,74923520.0,0,1


In [293]:
safe_dataframe.head(20)

,Player Initial,Dealer Hand,Has Ace,Action,Earning,Money,Result,Correct Action
0,8,9,0,1,-5.0,74999995.0,0,0
1,13,9,1,0,-5.0,74999990.0,0,1
2,14,6,0,0,-5.0,74999985.0,0,1
3,20,9,0,0,5.0,74999990.0,1,0
4,11,9,0,1,5.0,74999995.0,1,1
5,9,10,0,1,-5.0,74999990.0,0,0
6,13,4,0,0,5.0,74999995.0,1,0
7,17,2,0,0,-5.0,74999990.0,0,1
8,8,7,0,1,5.0,74999995.0,1,1
9,14,10,1,0,5.0,75000000.0,1,0


In [294]:
random_dataframe = pd.DataFrame()
random_dataframe['Player Initial'] = random_player_initial_sum
random_dataframe['Dealer Hand'] = random_dealer_first_card
random_dataframe['Has Ace'] = random_has_ace
random_dataframe['Action'] = random_action
random_dataframe['Earning'] = random_earning
random_dataframe['Money'] = random_save_money
random_dataframe['Result'] = random_result
random_dataframe['Correct Action'] = random_correct_action

In [295]:
# Player Initial is player starting sum 
# Dealer hand is the first card they get. 
# Action varies. (0: no action, 1: hit, 2: stay, 3: double down)
# Earning varies. Anything negative is a loss, 0's is a push, and anything + is a win
# Money Keeps count of the money. They start with $1000 
# Result varies. (0: lose, 1: win, 2: tie)
random_dataframe

,Player Initial,Dealer Hand,Has Ace,Action,Earning,Money,Result,Correct Action
0,20,5,0,0,5.0,75000005.0,1,0
1,18,3,1,0,5.0,75000010.0,1,0
2,13,10,0,1,-5.0,75000005.0,0,0
3,11,10,0,0,-5.0,75000000.0,0,1
4,18,5,0,1,-5.0,74999995.0,0,0
...,...,...,...,...,...,...,...,...
169995,18,11,0,0,-5.0,74809637.5,0,1
169996,16,7,0,0,-5.0,74809632.5,0,1
169997,16,6,1,1,5.0,74809637.5,1,1
169998,12,10,0,1,5.0,74809642.5,1,1


In [296]:
random_dataframe.head(20)

,Player Initial,Dealer Hand,Has Ace,Action,Earning,Money,Result,Correct Action
0,20,5,0,0,5.0,75000005.0,1,0
1,18,3,1,0,5.0,75000010.0,1,0
2,13,10,0,1,-5.0,75000005.0,0,0
3,11,10,0,0,-5.0,75000000.0,0,1
4,18,5,0,1,-5.0,74999995.0,0,0
5,16,10,0,1,-5.0,74999990.0,0,0
6,12,10,0,1,-5.0,74999985.0,0,0
7,20,10,1,1,-5.0,74999980.0,0,0
8,14,8,0,0,-5.0,74999975.0,0,1
9,19,10,0,1,5.0,74999980.0,1,1


In [297]:
# Merging both the safe_dataframe and random_dataframe
both_gameplay_dataset = [safe_dataframe, random_dataframe]
#merge_dataframe = pd.concat(both_gameplay_dataset)
merge_dataframe = random_dataframe
#merge_dataframe = safe_dataframe

In [298]:
# Remove our Results and money. This will help train 
Training_Set = merge_dataframe.drop(['Money', "Earning", "Result", "Correct Action"], axis = 1) 
Training_Set

,Player Initial,Dealer Hand,Has Ace,Action
0,20,5,0,0
1,18,3,1,0
2,13,10,0,1
3,11,10,0,0
4,18,5,0,1
...,...,...,...,...
169995,18,11,0,0
169996,16,7,0,0
169997,16,6,1,1
169998,12,10,0,1


In [299]:
Correct_Action_Set = merge_dataframe['Correct Action']
Correct_Action_Set

0         0
1         0
2         0
3         1
4         0
         ..
169995    1
169996    1
169997    1
169998    1
169999    1
Name: Correct Action, Length: 170000, dtype: int64

In [300]:
x_train, x_test, y_train, y_test = train_test_split(Training_Set, Correct_Action_Set, test_size = 0.20, random_state = 37)
x_train # This will display the 80% of the randomized rows that will be used to train the ANN

,Player Initial,Dealer Hand,Has Ace,Action
14541,18,8,0,0
42048,18,10,0,0
18549,16,9,0,1
69904,16,8,0,0
140152,13,3,0,0
...,...,...,...,...
120483,9,3,0,0
160181,17,3,0,1
162043,20,11,0,0
20843,19,7,0,1


In [301]:
x_test

,Player Initial,Dealer Hand,Has Ace,Action
58629,12,10,0,1
25247,19,6,1,0
111129,20,10,0,1
7291,21,4,1,0
62842,11,7,0,1
...,...,...,...,...
155650,18,10,0,0
737,13,11,0,0
18865,18,10,0,1
126629,10,10,0,0


In [302]:
scaler = MinMaxScaler()
# Transforming the data for x_train using MinMaxScaler
#x_train_scaler = scaler.fit_transform(x_train)
x_train_initial = x_train['Player Initial']
x_train_dealer = x_train['Dealer Hand']
x_train_ace = x_train['Has Ace']
x_train_action = x_train['Action']
#x_train_scaler = np.array([x_train_initial,x_train_dealer,x_train_ace,x_train_action]).reshape(1,-1)
x_train_scaler = np.array([x_train]).reshape(1,-1)
print(len(x_train_scaler))
print(x_train_scaler)

1
[[18  8  0 ...  7  0  0]]


In [303]:
scaler = MinMaxScaler()
# Transforming the data for x_test using MinMaxScaler
#x_test_scaler = scaler.fit_transform(x_test)
x_test_scaler = np.array(x_test).reshape(1,-1)
x_test_scaler

array([[12, 10,  0, ..., 10,  0,  0]])

In [304]:
y_train #= np.array(y_train)#.reshape(1,-1)
print(len(y_train))
print(y_train)

136000
14541     0
42048     0
18549     0
69904     1
140152    1
         ..
120483    1
160181    0
162043    1
20843     0
165775    1
Name: Correct Action, Length: 136000, dtype: int64


In [305]:
blackjack_AI = keras.Sequential(
    [
        layers.Dense(units = 23, activation = 'sigmoid', kernel_initializer = 'he_uniform', name = 'sigmoid_layer1'),
        layers.Dense(units = 21, activation = 'sigmoid',  name = 'sigmoid_layer2'),
        layers.Dense(units = 19, activation = 'relu',  name = 'relu_layer3'),
        layers.Dense(units = 17, activation = 'relu', kernel_initializer = 'he_uniform', name = 'relu_layer4'),
        layers.Dense(units = 15, activation = 'relu', kernel_initializer = 'he_uniform', name = 'relu_layer5'),
        layers.Dense(units = 13, activation = 'sigmoid', kernel_initializer = 'he_uniform', name = 'sigmoid_layer6'),
        layers.Dense(units = 11, activation = 'relu', name = 'relu_layer7'),
        layers.Dense(units = 9, activation = 'sigmoid', name = 'sigmoid_layer8'),
        layers.Dense(units = 7, activation = 'sigmoid', kernel_initializer = 'he_uniform', name = 'sigmoid_layer9'),
        layers.Dense(units = 5, activation = 'relu', kernel_initializer = 'he_uniform', name = 'relu_layer10'),
        layers.Dense(units = 3, activation = 'sigmoid', name = 'sigmoid_layer11'),
        layers.Dense(units = 1, activation = 'sigmoid', name = 'sigmoid_layer12')
    ] 
)  

In [306]:
# Now we will take the data from our safe_play and our random_play to train our model. 
#blackjack_AI = Sequential()
#blackjack_AI.add(Dense(16))
#blackjack_AI.add(Dense(100))
#blackjack_AI.add(Dense(32))
#blackjack_AI.add(Dense(8))
#blackjack_AI.add(Dense(3))
#blackjack_AI.add(Dense(1, activation='sigmoid')) # The output neurons are subject to change depending on number of classes found.
optimizer = keras.optimizers.Adam(learning_rate=0.0000001)
#blackjack_AI.compile(optimizer = optimizer, loss="binary_crossentropy", metrics=["accuracy"])
blackjack_AI.compile(optimizer = 'adam', loss="binary_crossentropy", metrics=["accuracy"])
blackjack_AI.fit(x = x_train, y = y_train, batch_size = 20, validation_split = 0.20, epochs = 10)

Epoch 1/10
5440/5440 [==============================] - 13s 2ms/step - loss: 0.7003 - accuracy: 0.5120 - val_loss: 0.6916 - val_accuracy: 0.5301
Epoch 2/10
5440/5440 [==============================] - 13s 2ms/step - loss: 0.6918 - accuracy: 0.5271 - val_loss: 0.6913 - val_accuracy: 0.5301
Epoch 3/10
5440/5440 [==============================] - 12s 2ms/step - loss: 0.6920 - accuracy: 0.5249 - val_loss: 0.6914 - val_accuracy: 0.5301
Epoch 4/10
5440/5440 [==============================] - 13s 2ms/step - loss: 0.6915 - accuracy: 0.5296 - val_loss: 0.6913 - val_accuracy: 0.5301
Epoch 5/10
5440/5440 [==============================] - 13s 2ms/step - loss: 0.6916 - accuracy: 0.5282 - val_loss: 0.6914 - val_accuracy: 0.5301
Epoch 6/10
5440/5440 [==============================] - 13s 2ms/step - loss: 0.6919 - accuracy: 0.5254 - val_loss: 0.6914 - val_accuracy: 0.5301
Epoch 7/10
5440/5440 [==============================] - 13s 2ms/step - loss: 0.6918 - accuracy: 0.5268 - val_loss: 0.6914 - val_ac

In [307]:
blackjack_AI.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sigmoid_layer1 (Dense)       (20, 23)                  115       
_________________________________________________________________
sigmoid_layer2 (Dense)       (20, 21)                  504       
_________________________________________________________________
relu_layer3 (Dense)          (20, 19)                  418       
_________________________________________________________________
relu_layer4 (Dense)          (20, 17)                  340       
_________________________________________________________________
relu_layer5 (Dense)          (20, 15)                  270       
_________________________________________________________________
sigmoid_layer6 (Dense)       (20, 13)                  208       
_________________________________________________________________
relu_layer7 (Dense)          (20, 11)                 

In [308]:
merge_dataframe

,Player Initial,Dealer Hand,Has Ace,Action,Earning,Money,Result,Correct Action
0,20,5,0,0,5.0,75000005.0,1,0
1,18,3,1,0,5.0,75000010.0,1,0
2,13,10,0,1,-5.0,75000005.0,0,0
3,11,10,0,0,-5.0,75000000.0,0,1
4,18,5,0,1,-5.0,74999995.0,0,0
...,...,...,...,...,...,...,...,...
169995,18,11,0,0,-5.0,74809637.5,0,1
169996,16,7,0,0,-5.0,74809632.5,0,1
169997,16,6,1,1,5.0,74809637.5,1,1
169998,12,10,0,1,5.0,74809642.5,1,1


In [309]:
# Evaluate the model by using the .predict function
#pred = blackjack_AI.predict(x_test_scaler)
pred = blackjack_AI.predict(x_test)
#output_action = np.argmax(pred, axis=1) # Since we used categories, we will pick one category with the highest probability of the classes. 
pred # Print the prediction.

array([[0.46056956],
       [0.46056956],
       [0.46056956],
       ...,
       [0.46056956],
       [0.46056956],
       [0.46056956]], dtype=float32)

In [310]:
# player initial, dealer hand, has ace, and action will be fed into our model.predict to output a correct action
output_action = blackjack_AI.predict(x_test)
round_output = output_action.round()
Display_Actual = sum(y_test == 1)
Display_predicted = sum(round_output == 1)
Display_Actual_stay = sum(y_test == 0)
Display_predicted_stay = sum(round_output == 0)
print("This is predicted hit:", Display_predicted)
print("This is actual hit:", Display_Actual)
print("This is predicted stay:", Display_predicted_stay)
print("This is actual stay:", Display_Actual_stay)
print(round_output)

This is predicted hit: [0]
This is actual hit: 16172
This is predicted stay: [34000]
This is actual stay: 17828
[[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]


In [311]:
print(classification_report(y_test,round_output))

              precision    recall  f1-score   support

           0       0.52      1.00      0.69     17828
           1       0.00      0.00      0.00     16172

    accuracy                           0.52     34000
   macro avg       0.26      0.50      0.34     34000
weighted avg       0.27      0.52      0.36     34000



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [312]:
new_test = pd.DataFrame()
card = [17]
hand = [5]
ace = [1]
acc = [0]
new_test['Player Initial'] = card
new_test['Dealer Hand'] = hand
new_test['Has Ace'] = ace
new_test['Action'] = acc
print(new_test)

input_array = np.array([5, 5, 0, 0]).reshape(1,-1)
print("LENGTH",len(input_array))
test = np.array(new_test).reshape(1,-1)
new_test_scaler = scaler.fit_transform(new_test)
print("ARRAY",input_array)
print("TST",test)
print("SCALER",new_test_scaler)
action = blackjack_AI.predict(new_test)
action = action.round()
print(action)

   Player Initial  Dealer Hand  Has Ace  Action
0              17            5        1       0
LENGTH 1
ARRAY [[5 5 0 0]]
TST [[17  5  1  0]]
SCALER [[0. 0. 0. 0.]]
[[0.]]


In [313]:
# Have the model we train play the blackjack game using model.predict()
computer_play = True
AI_play_money = 7500000
update_money = 0
AI_profit_or_loss = 0
count = 1
int(AI_play_money)
deck_cards = shuffle()
for i in range(0,100000):#170000,1):
  if(AI_play_money < 5):
    break
  if(AI_play_money > AI_play_money*2):
    break
  if(i+1 == (count * 10000)):
    count += 1
    print("Game round:",i+1)
  #print("Game round:", i+1, "__________________________________________________")
  update_money, deck_cards = model_AI_game(AI_play_money, deck_cards, computer_play)
  #print("Here")
  #print("\nPrevious $", safe_play_money)
  #print("Player $", update_money)
  AI_profit_or_loss = update_money - AI_play_money
  AI_play_money = update_money
  AI_save_money.append(update_money)
  AI_earning.append(AI_profit_or_loss)
  #print("This is profit or loss:",safe_profit_or_loss)

Game round: 10000
Game round: 20000
Game round: 30000
Game round: 40000
Game round: 50000
Game round: 60000
Game round: 70000
Game round: 80000
Game round: 90000
Game round: 100000


In [314]:
AI_dataframe = pd.DataFrame()
AI_dataframe['Player Initial'] = AI_initial_sum
AI_dataframe['Dealer Hand'] = AI_dealer_first_card
AI_dataframe['Has Ace'] = AI_has_ace
AI_dataframe['Possible Action'] = AI_possible_action
AI_dataframe['Earning'] = AI_earning
AI_dataframe['Money'] = AI_save_money
AI_dataframe['Result'] = AI_result
AI_dataframe['Actual Action'] = AI_actual_action
AI_dataframe

,Player Initial,Dealer Hand,Has Ace,Possible Action,Earning,Money,Result,Actual Action
0,12,11,0,1,-5.0,7499995.0,0,0
1,8,7,0,0,-5.0,7499990.0,0,0
2,12,7,0,1,-5.0,7499985.0,0,0
3,13,8,0,1,5.0,7499990.0,1,0
4,15,6,0,0,-5.0,7499985.0,0,0
...,...,...,...,...,...,...,...,...
99995,13,8,0,1,-5.0,7421505.0,0,0
99996,19,10,0,0,-5.0,7421500.0,0,0
99997,15,10,0,0,-5.0,7421495.0,0,0
99998,13,9,0,0,-5.0,7421490.0,0,0


In [315]:
AI_dataframe.head(60)

,Player Initial,Dealer Hand,Has Ace,Possible Action,Earning,Money,Result,Actual Action
0,12,11,0,1,-5.0,7499995.0,0,0
1,8,7,0,0,-5.0,7499990.0,0,0
2,12,7,0,1,-5.0,7499985.0,0,0
3,13,8,0,1,5.0,7499990.0,1,0
4,15,6,0,0,-5.0,7499985.0,0,0
5,14,3,0,1,5.0,7499990.0,1,0
6,14,10,0,0,-5.0,7499985.0,0,0
7,20,10,0,0,5.0,7499990.0,1,0
8,19,10,0,1,5.0,7499995.0,1,0
9,8,4,0,0,-5.0,7499990.0,0,0


In [316]:
get_action_only = AI_dataframe['Actual Action']
get_action_only

sum_action = sum(get_action_only == 1)
print("Total sum of hit is", sum_action)

Total sum of hit is 0


In [317]:
safe_win = 0
safe_lose = 0
safe_tie = 0
get_safe_game_played = len(safe_result)
for i in range(0,get_safe_game_played):
  if(safe_result[i] == 0):
    safe_lose += 1
  elif(safe_result[i] == 1):
    safe_win += 1
  elif(safe_result[i] == 2):
    safe_tie += 1

round_win = round((safe_win / get_safe_game_played),2) * 100
round_lose = round((safe_lose / get_safe_game_played),2) * 100
round_tie = round((safe_tie / get_safe_game_played),2) * 100

print("Statistics using safe action strategy")
print("Win percentage:", str(round_win) + "%")
print("Lose percentage:", str(round_lose) + "%")
print("Tie percentage:", str(round_tie) + "%")
print()


###########################################################################################################################################

random_win = 0
random_lose = 0
random_tie = 0
get_random_game_played = len(random_result)
for i in range(0,get_random_game_played):
  if(random_result[i] == 0):
    random_lose += 1
  elif(random_result[i] == 1):
    random_win += 1
  elif(random_result[i] == 2):
    random_tie += 1

round_win = round((random_win / get_random_game_played),2) * 100
round_lose = round((random_lose / get_random_game_played),2) * 100
round_tie = round((random_tie / get_random_game_played),2) * 100

print("Statistics using random action strategy")
print("Win percentage:", str(round_win) + "%")
print("Lose percentage:", str(round_lose) + "%")
print("Tie percentage:", str(round_tie) + "%")
print()

##############################################################################################################################################

AI_win = 0
AI_lose = 0
AI_tie = 0
get_AI_game_played = len(AI_result)
for i in range(0,get_AI_game_played):
  if(AI_result[i] == 0):
    AI_lose += 1
  elif(AI_result[i] == 1):
    AI_win += 1
  elif(AI_result[i] == 2):
    AI_tie += 1

round_win = round((AI_win / get_AI_game_played),2) * 100
round_lose = round((AI_lose / get_AI_game_played),2) * 100
round_tie = round((AI_tie / get_AI_game_played),2) * 100

print("Statistics using AI action strategy")
print("Win percentage:", str(round_win) + "%")
print("Lose percentage:", str(round_lose) + "%")
print("Tie percentage:", str(round_tie) + "%")
print()

Statistics using safe action strategy
Win percentage: 41.0%
Lose percentage: 53.0%
Tie percentage: 6.0%

Statistics using random action strategy
Win percentage: 35.0%
Lose percentage: 60.0%
Tie percentage: 5.0%

Statistics using AI action strategy
Win percentage: 39.0%
Lose percentage: 56.99999999999999%
Tie percentage: 5.0%

